In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [6]:
# Cargar datos
raw_data_17 = pd.read_csv('PROYECTO2_BI/Data/nac2017.csv')
raw_data_18 = pd.read_csv('PROYECTO2_BI/Data/nac2018.csv')
raw_data_21 = pd.read_csv('PROYECTO2_BI/Data/nac2021.csv')
raw_data_22 = pd.read_csv('PROYECTO2_BI/Data/nac2022.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'PROYECTO2_BI/Data/nac2017.csv'

In [ ]:
raw_data_21 = raw_data_21.drop(columns="TIPOFORMULARIO")

In [ ]:
# Muestra de los datos
raw_data.head()

In [ ]:
# Información de los datos
raw_data.info()

Descripción variables numéricas:

In [ ]:
# Descripción variables numéricas
pd.set_option('display.max_columns', None) #Si hay muchas columnas
raw_data.describe()

In [ ]:
# Gráficos para variables numéricas
features_num = raw_data.select_dtypes(include=['float64', 'int64']).columns

plt.figure(figsize=(20, 15)) 

for i, feature in enumerate(features_num):
    plt.subplot(2, 4, i+1)  # Crear subplots en una cuadrícula de 2 filas y 4 columnas
    sns.histplot(data=raw_data, x=feature, kde=False, bins=10)
    plt.title(f'Histograma de {feature}')
    plt.tight_layout()

plt.show()

Descripción de variables categóricas:

In [ ]:
# Descripción de Variables categóricas
features_cat = raw_data.select_dtypes(include=['object']).columns

for i, feature in enumerate(features_cat):
    print(pd.Series(raw_data[feature]).value_counts())

In [ ]:
# Gráficos para variables categporicas

plt.figure(figsize=(20, 15)) 

for i, feature in enumerate(features_cat):
    plt.subplot(2, 3, i+1)  # Crear subplots en una cuadrícula de 2 filas y 3 columnas (ajustado para 5 variables)
    sns.countplot(data=raw_data, x=feature, palette='viridis', hue=feature)
    plt.title(f'Gráfico de barras de {feature}')
    plt.tight_layout()

plt.show()

Distribución de la variable objetivo:

In [ ]:
# Lineal
plt.figure(figsize=(10, 6))
sns.histplot(raw_data["Variable_Objetivo"], bins=30, kde=True)
plt.title("Distribución de la variable objetivo ( )")
plt.xlabel("Variable objetivo")
plt.ylabel("Frecuencia")
plt.show()

# Categórica
sns.countplot(x='Evento', data=raw_data)
plt.title("Distribución de la variable objetivo ( )")
plt.show()

### 2.2. Calidad de datos

**Consistencia:** hablar de si los tipos de datos encontrados son consistentes con los del diccionario de datos. Los datos deben ser coherentes con las descripciones recibidas.

**Unicidad:** Hablar de registros duplicados. Cada registro debe ser único, y la presencia de datos duplicados indica un problema de calidad. 

**Completitud:** Hablar de valores faltantes. Es importante que toda la información necesaria esté disponible y no haya campos obligatorios vacíos o incompletos.

**Validez:** Hablar de valores atípicos. Los datos deben cumplir con las reglas del negocio o estar dentro de rangos válidos.
Ejemplo: Si en una columna de edad se encuentran valores negativos o extremadamente altos (como 200 años), estos serían considerados valores atípicos o no válidos.

#### Registros duplicados:

In [ ]:
duplicates = raw_data.duplicated(keep = False).sum()
print(f'Registros duplicados en el conjunto de datos: {duplicates}')

#### Valores faltantes:

In [ ]:
missing_values = raw_data.isnull().sum()
print("\nValores faltantes en el conjunto de datos:\n", missing_values)

# Porcentajes 
missing_values_p = (missing_values / len(raw_data)) * 100
print("\Porcentaje de valores faltantes en el conjunto de datos:\n", missing_values_p)

# Visualización de los valores faltantes
sns.heatmap(raw_data.isnull(), cbar=False, cmap='viridis')
plt.title("Mapa de calor de los valores faltantes")
plt.show()

#### Valores atípicos:

In [ ]:
# Contar valores negativos en cada columna numérica
negative_counts = (raw_data[features_num] < 0).sum()

print("Cantidad de valores negativos en cada columna numérica:")
print(negative_counts)

#### Outliers:

In [ ]:
# Porcentaje de outliers
features_outliers = []
print('Número y porcentaje de outliers:')

for column in features_num:
    Q1 = raw_data[column].quantile(0.25)
    Q3 = raw_data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = raw_data[(raw_data[column] < lower_bound) | (raw_data[column] > upper_bound)]
    percentage = (len(outliers) / len(raw_data[column])) * 100
    
    if percentage > 0:
        features_outliers.append(column)
        
    print(f'{column}: {len(outliers)} | {percentage:.2f}%')

In [ ]:
# Distribución de variables con outliers
plt.figure(figsize=(4 * len(features_outliers), 5))

for i, column in enumerate(features_outliers, start=1):
    plt.subplot(1, len(features_outliers), i)
    sns.boxplot(y=raw_data[column])
    plt.title(f'Boxplot de {column}')

plt.tight_layout()
plt.show()